# Laboratorio 2 - IA

## Autores

Nelson Escalante

Milton Polanco

## Problema 1

In [1]:
import numpy as np

# Array con los datos a clasificar, 
# con columnas X de 1 a 4 y filas Y de 0 a 1
joint = np.array([
    [0.09, 0.16, 0.17, 0.05],   # Y = 0
    [0.15, 0.07, 0.10, 0.21]    # Y = 1
])

# Clasificador bayesiano optimo
classifier = np.argmax(joint, axis=0)

# Error bayesiano
bayes_error = np.sum(np.min(joint, axis=0))

print("Clasificion optima por X:")
for i, y in enumerate(classifier, start=1):
    print(f"X = {i} -> Y = {y}")

print("\nError del clasificador:", bayes_error)

Clasificion optima por X:
X = 1 -> Y = 1
X = 2 -> Y = 0
X = 3 -> Y = 0
X = 4 -> Y = 1

Error del clasificador: 0.31


## Problema 2

In [3]:
x_star = 0.5 * np.log(3)

Pe = 0.5 * (np.exp(-3 * x_star) + (1 - np.exp(-x_star)))

print("Umbral x* =", x_star)
print("Error Bayesiano =", Pe)

Umbral x* = 0.5493061443340549
Error Bayesiano = 0.3075499102701248
